In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
df = pd.read_csv('../Dataset/processed_data/final_dataset.csv')

descriptions = df['description']  
domains = df['domain'] 
sub_domains = df['sub_domain']  

In [3]:

# Tokenize descriptions
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(descriptions)
X = tokenizer.texts_to_sequences(descriptions)
X = pad_sequences(X, padding='post', maxlen=100)  # Adjust maxlen as needed


In [4]:

# Encode labels (domain and sub_domain)
domain_encoder = LabelEncoder()
sub_domain_encoder = LabelEncoder()
y_domain = domain_encoder.fit_transform(domains)
y_sub_domain = sub_domain_encoder.fit_transform(sub_domains)

# One-hot encode labels
y_domain = tf.keras.utils.to_categorical(y_domain, num_classes=len(domain_encoder.classes_))
y_sub_domain = tf.keras.utils.to_categorical(y_sub_domain, num_classes=len(sub_domain_encoder.classes_))


# Split data into train (80%) and validation (20%)
X_train, X_val, y_domain_train, y_domain_val, y_sub_train, y_sub_val = train_test_split(
    X, y_domain, y_sub_domain, test_size=0.2, random_state=42
)

In [5]:

# Build Neural Network Model
input_layer = Input(shape=(X.shape[1],))
embedding_layer = Embedding(input_dim=10000, output_dim=128, input_length=X.shape[1])(input_layer)
lstm_layer = LSTM(64, return_sequences=False)(embedding_layer)
dropout_layer = Dropout(0.5)(lstm_layer)
dense_layer = Dense(64, activation='relu')(dropout_layer)

# Domain Output Layer
domain_output = Dense(len(domain_encoder.classes_), activation='softmax', name='domain')(dense_layer)

# Sub-Domain Output Layer
sub_domain_output = Dense(len(sub_domain_encoder.classes_), activation='softmax', name='sub_domain')(dense_layer)

# Create the model
model = Model(inputs=input_layer, outputs=[domain_output, sub_domain_output])


d:\Jay\CDAC\Project\Domain Classifier\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [6]:
# Compile the model with separate metrics for each output
model.compile(optimizer='adam',
              loss=['categorical_crossentropy', 'categorical_crossentropy'],
              metrics=[['accuracy'], ['accuracy']])  

# Summary of the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 100, 128)  │  1,280,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 64)        │     49,408 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      4,160 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ domain (Dense)      │ (None, 3)         │        195 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sub_domain (Dense)  │ (None, 42)        │      2,730 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,336,493 (5.10 MB)

 Trainable params: 1,336,493 (5.10 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:

# Train the model
history = model.fit(
    X_train, [y_domain_train, y_sub_train],
    epochs=20,
    batch_size=16,
    validation_data=(X_val, [y_domain_val, y_sub_val]),
    verbose=1
)

Epoch 1/20
5260/5260 ━━━━━━━━━━━━━━━━━━━━ 208s 39ms/step - domain_accuracy: 0.8460 - domain_loss: 0.3823 - loss: 2.5950 - sub_domain_accuracy: 0.2308 - sub_domain_loss: 2.2127 - val_domain_accuracy: 0.9672 - val_domain_loss: 0.0930 - val_loss: 1.6820 - val_sub_domain_accuracy: 0.4113 - val_sub_domain_loss: 1.5889
Epoch 2/20
5260/5260 ━━━━━━━━━━━━━━━━━━━━ 208s 40ms/step - domain_accuracy: 0.9759 - domain_loss: 0.0718 - loss: 1.6182 - sub_domain_accuracy: 0.4145 - sub_domain_loss: 1.5465 - val_domain_accuracy: 0.9875 - val_domain_loss: 0.0420 - val_loss: 1.1295 - val_sub_domain_accuracy: 0.6025 - val_sub_domain_loss: 1.0875
Epoch 3/20
5260/5260 ━━━━━━━━━━━━━━━━━━━━ 240s 46ms/step - domain_accuracy: 0.9911 - domain_loss: 0.0308 - loss: 1.0833 - sub_domain_accuracy: 0.6161 - sub_domain_loss: 1.0525 - val_domain_accuracy: 0.9878 - val_domain_loss: 0.0419 - val_loss: 0.9342 - val_sub_domain_accuracy: 0.7082 - val_sub_domain_loss: 0.8923
Epoch 4/20
5260/5260 ━━━━━━━━━━━━━━━━━━━━ 208s 39ms/ste

In [8]:
# Evaluate the model on validation set
val_loss, val_domain_loss, val_sub_loss, val_domain_acc, val_sub_acc = model.evaluate(
    X_val, [y_domain_val, y_sub_val])
print(
    f"Validation Accuracy - Domain: {val_domain_acc:.4f}, Sub-Domain: {val_sub_acc:.4f}")

658/658 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - domain_accuracy: 0.9865 - domain_loss: 0.1062 - loss: 1.4629 - sub_domain_accuracy: 0.7759 - sub_domain_loss: 1.3567
Validation Accuracy - Domain: 0.9869, Sub-Domain: 0.7796


In [11]:
description_input = ["""Battery Details- Type: Tall tubular batteryCapacity : 200Ah/12V: Construction: rugged construction
Warranty -66 months ( 48 FOC+18Pro Rata)
Dimension (in cm) - 512x192x466
Weight- 64.8K.G"""]
description_seq = tokenizer.texts_to_sequences(description_input)
description_padded = pad_sequences(description_seq, padding='post', maxlen=100)

domain_pred, sub_domain_pred = model.predict(description_padded)
domain_pred_label = domain_encoder.inverse_transform(
    domain_pred.argmax(axis=1))
sub_domain_pred_label = sub_domain_encoder.inverse_transform(
    sub_domain_pred.argmax(axis=1))

print(f"Predicted Domain: {domain_pred_label}")
print(f"Predicted Sub-Domain: {sub_domain_pred_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Predicted Domain: ['Ecommerce']
Predicted Sub-Domain: ['Household']


In [13]:
# Save the entire model (architecture + weights + optimizer state)
import pickle

# Save the model
model.save("../models/neural_network/multi_label_model.keras")

# Save the tokenizer
with open("../models/neural_network/tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

# Save Label Encoders
with open("../models/neural_network/domain_encoder.pkl", "wb") as f:
    pickle.dump(domain_encoder, f)

with open("../models/neural_network/sub_domain_encoder.pkl", "wb") as f:
    pickle.dump(sub_domain_encoder, f)

print("Model and necessary objects saved successfully! 🎯")

Model and necessary objects saved successfully! 🎯


In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
# Load the saved model
model = load_model("../models/neural_network/multi_label_model.h5")

# Load Tokenizer
with open("../models/neural_network/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

# Load Label Encoders
with open("../models/neural_network/domain_encoder.pkl", "rb") as f:
    domain_encoder = pickle.load(f)

with open("../models/neural_network/sub_domain_encoder.pkl", "rb") as f:
    sub_domain_encoder = pickle.load(f)

print("Model and necessary objects loaded successfully! 🚀")

### Hyperparameter Tuning

In [5]:
# Import Keras Tuner
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from keras_tuner import HyperModel, Hyperband
import keras_tuner

In [6]:

# Define HyperModel for Tuning
class TextClassifierHyperModel(HyperModel):
    def build(self, hp):
        input_layer = Input(shape=(X.shape[1],))

        # Tune Embedding Layer size
        embedding_dim = hp.Int(
            'embedding_dim', min_value=64, max_value=256, step=64)
        embedding_layer = Embedding(
            input_dim=10000, output_dim=embedding_dim, input_length=X.shape[1])(input_layer)

        # Tune LSTM Units
        lstm_units = hp.Int('lstm_units', min_value=32, max_value=128, step=32)
        lstm_layer = LSTM(lstm_units, return_sequences=False)(embedding_layer)

        # Tune Dropout Rate
        dropout_rate = hp.Float(
            'dropout_rate', min_value=0.2, max_value=0.5, step=0.1)
        dropout_layer = Dropout(dropout_rate)(lstm_layer)

        # Tune Dense Layer Units
        dense_units = hp.Int('dense_units', min_value=32,
                             max_value=128, step=32)
        dense_layer = Dense(dense_units, activation='relu')(dropout_layer)

        # Domain Output Layer
        domain_output = Dense(len(domain_encoder.classes_),
                              activation='softmax', name='domain')(dense_layer)

        # Sub-Domain Output Layer
        sub_domain_output = Dense(len(
            sub_domain_encoder.classes_), activation='softmax', name='sub_domain')(dense_layer)

        # Define Model
        model = Model(inputs=input_layer, outputs=[
                      domain_output, sub_domain_output])

        # Tune Learning Rate
        learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        optimizer = Adam(learning_rate=learning_rate)

        model.compile(optimizer=optimizer,
                      loss=['categorical_crossentropy',
                            'categorical_crossentropy'],
                      metrics=['accuracy', 'accuracy'])
        return model



In [ ]:

# Initialize Tuner
tuner = Hyperband(TextClassifierHyperModel(),
                  objective=keras_tuner.Objective(
                      "val_sub_domain_accuracy", direction="max"),
                  max_epochs=10,
                  factor=3,
                  directory='hyperband_tuning',
                  project_name='text_classifier')

d:\Jay\CDAC\Project\Domain Classifier\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:

# Search for best hyperparameters
tuner.search(X_train, [y_domain_train, y_sub_train],
             epochs=10,
             batch_size=16,
             validation_data=(X_val, [y_domain_val, y_sub_val]),
             verbose=1)


In [ ]:

# Get Best Hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Best Hyperparameters Found:
Embedding Size: {best_hps.get('embedding_dim')}
LSTM Units: {best_hps.get('lstm_units')}
Dropout Rate: {best_hps.get('dropout_rate')}
Dense Units: {best_hps.get('dense_units')}
Learning Rate: {best_hps.get('learning_rate')}
""")


In [ ]:

# Train Best Model
best_model = tuner.hypermodel.build(best_hps)


In [ ]:

history = best_model.fit(
    X_train, [y_domain_train, y_sub_train],
    epochs=20,
    batch_size=16,
    validation_data=(X_val, [y_domain_val, y_sub_val]),
    verbose=1
)


In [ ]:
# Save the trained model
best_model.save("../models/neural_network/best_model.keras")

In [ ]:

# Evaluate Model on Validation Data
val_loss, val_domain_loss, val_sub_loss, val_domain_acc, val_sub_acc = best_model.evaluate(
    X_val, [y_domain_val, y_sub_val])
print(
    f"Validation Accuracy - Domain: {val_domain_acc:.4f}, Sub-Domain: {val_sub_acc:.4f}")